# Empowerment: Pacman

In [1]:
import numpy as np
from itertools import combinations_with_replacement, combinations, permutations, product
import pygame
import random
import copy
import math

background_color = [0, 0, 0]
pacman_color = [255, 255, 0]
wall_color = [255, 255, 255]
ghost_color = [0, 0, 255]
blocksize = 26
agentsize = 10
ghostsize = 20

In [2]:
# ' ' - nothing
# 'P' - player
# '#' - wall
# 'G' - ghost

mfield = np.array(
        [[' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', '#', '#', '#', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#', ' '],
         [' ', ' ', '#', ' ', '#', ' ', ' ', '#', ' ', ' ', ' ', ' ', 'G', '#', ' '],
         ['#', '#', '#', ' ', '#', ' ', ' ', ' ', 'P', ' ', ' ', ' ', ' ', '#', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', '#', '#', '#', '#', ' ', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', 'G', ' ', ' ', ' ', ' ', '#', ' ', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', '#', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' '],
         [' ', ' ', '#', '#', '#', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', '#', '#', '#', '#', ' ', ' ', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'G', ' ']])

mfield = np.array(
        [[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', ' ', 'G', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', 'P', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'G', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', ' ', ' ', 'G', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', 'G', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', 'G', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'G', ' ']])

height = len(mfield)
width = len(mfield[0])

In [3]:
paths = set()
for i in product(range(4), repeat=4):
    paths.add(i)

In [4]:
# Returns the resulting position of a moving object, given its current position and its action
def update_position(position, action):
    newpos = [position[0] + action[0], position[1] + action[1]]
    newpos[0] %= height
    newpos[1] %= width
    return newpos

In [5]:
def player_position(field):
    return np.array(np.where(field == 'P')).flatten()

In [6]:
# Returns a list of possible actions (lists with relative y and x positions), given the current field and angent position
def possible_actions(field, position):
    actions = []
    for action in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
        newpos = update_position(position, action)
        target = field[newpos[0]][newpos[1]]
        if target == '#':
            continue
        elif target == 'G':
            continue
        actions.append(action)
    return actions

In [7]:
# Updates a field matrix and moves an agent from the old position to a new position
def move_agent(field, old_position, new_position):
    nfield = copy.deepcopy(field)
    agent = nfield[old_position[0]][old_position[1]]
    target = nfield[new_position[0]][new_position[1]]
    if not (agent == 'P' and target == 'G'):
        nfield[new_position[0]][new_position[1]] = agent
    nfield[old_position[0]][old_position[1]] = ' '
    return nfield

#print(move_agent(field, [5, 3], [3, 3]))

In [8]:
def update(field, player_action):
    def euclidean(p_x, p_y, g_x, g_y):
        return math.sqrt((g_x - p_x)**2 + (g_y - p_y)**2)
    
    ## calculation of ghost actions 
    # ghost positions
    ghost_x = []
    ghost_y = []
    ghost_x, ghost_y = np.where(field == 'G')
    
    player_pos = player_position(field)

    for ghost_pos in zip(ghost_x, ghost_y):
        actions_ghost = possible_actions(field, ghost_pos)
        distance = 10000 # high value
        best_pos = ghost_pos
        for action in actions_ghost:
            possible_pos = update_position(ghost_pos, action)
            curr_distance = euclidean(player_pos[0], player_pos[1], possible_pos[0], possible_pos[1])
            
            if curr_distance < distance: 
                distance = curr_distance
                best_pos = possible_pos
        field = move_agent(field, ghost_pos, best_pos)
        
    new_player_pos = update_position(player_pos, player_action)
    field = move_agent(field, player_pos, new_player_pos)
    player_pos = new_player_pos
    return field

In [9]:
# Returns the empowerment resulting from traking a specific action in a field
def empowerment(field, action, steps):
    if steps == 0:
        return 1
    newfield = update(field, action)
    playerpos = player_position(newfield)
    if len(playerpos) != 2:
        return 1
    E = 1
    for nextaction in possible_actions(newfield, playerpos):
        E += empowerment(newfield, nextaction, steps - 1)
    return E

In [10]:
pygame.init()
clock = pygame.time.Clock()

size = [width * blocksize, height * blocksize]
screen = pygame.display.set_mode(size)
pygame.display.set_caption("Pacman")

done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
 
    screen.fill(background_color)
    for x in range(0, width):
        for y in range(0, height):
            if(mfield[y][x] == '#'):
                pygame.draw.rect(screen, wall_color, [x * blocksize, y * blocksize, blocksize - 1, blocksize - 1])
            elif(mfield[y][x] == 'P'):
                pygame.draw.circle(screen, pacman_color, [int(x * blocksize + blocksize / 2), int(y * blocksize + blocksize / 2)], agentsize)
            elif(mfield[y][x] == 'G'):
                pygame.draw.rect(screen, ghost_color, [int(x * blocksize + (blocksize - ghostsize) / 2),
                                                       int(y * blocksize + (blocksize - ghostsize) / 2),
                                                       ghostsize - 1, ghostsize - 1])            
    pygame.display.flip()
    
    if len(player_position(mfield)) == 2:
        agent_actions = possible_actions(mfield, player_position(mfield))
        best_empowerment = 0
        best_action = random.choice(agent_actions)
        for action in agent_actions:
            E = empowerment(mfield, action, 5)
            print(E)
            if(E > best_empowerment):
                best_empowerment = E
                best_action = action
        mfield = update(mfield, best_action)
    
    clock.tick(2)
    
pygame.quit()

697
1083
1115
799
528
1052
759
679
441
957
721
758
958
326
858
669
362
956
648
725
949
326
858
669
361
894
648
717
741
326
804
653
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
838
255
797
541
304
804
541
742
